# Install components

In [ ]:
%pip install  networkx
%pip install pandas
%pip install matplotlib
%pip install pyvis
import pandas as pd  
import networkx as nx  
import matplotlib.pyplot as plt  
from pyvis.network import Network  
import seaborn as sns

## Load dataset and create nodi for nodes and archi for edges, also create the graph, and calculate the metrics and visualize the graph whitl all componets

In [ ]:
# import the cvs
node_file = "data/marvel-unimodal-nodes.csv"
edges_file = "data/marvel-unimodal-edges.csv"

# read the csv
nodi = pd.read_csv(node_file)
archi = pd.read_csv(edges_file)

# check if the data are read
print("Esempio di dati dei nodi:")
print(nodi.head())
print("\nEsempio di dati degli archi:")
print(archi.head())

# 2. Creare il grafo
G = nx.Graph()  # can use DiGraph for oriented graph

# add node
for _, row in nodi.iterrows():
    G.add_node(row['Id'], label=row['Label'])

# Add arch
for _, row in archi.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['Weight'])

#number of node and edges
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print(f"\nNumero di nodi: {num_nodes}")
print(f"Numero di archi: {num_edges}")

# degree for each node
degree = dict(G.degree())
print("\nGrado dei nodi:")
print(degree)

# centrality degree
degree_centrality = nx.degree_centrality(G)
print("\nCentralità di grado:")
print({k: round(v, 2) for k, v in degree_centrality.items()})

# centrality: Betweenness
centrality_betweenness = nx.betweenness_centrality(G)
print("\nCentralità di betweenness:")
print({k: round(v, 2) for k, v in centrality_betweenness.items()})

# 4. Visualizzazione del grafo
plt.figure(figsize=(20, 15))
pos = nx.spring_layout(G)  # Posizione dei nodi
nx.draw_networkx_nodes(G, pos, node_size=500, node_color="skyblue")
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)
nx.draw_networkx_labels(G, pos, labels=nx.get_node_attributes(G, 'label'), font_size=10)
plt.title("Visualizzazione del Grafo")
plt.show()


## Filter the graph and visualize the subgraph with nodes with degree >= X

In [ ]:
nodes_high_degree = [n for n, d in G.degree() if d >= 120]
subnet = G.subgraph(nodes_high_degree)

# plot only the subnet
plt.figure(figsize=(20, 15))
pos = nx.spring_layout(subnet)
nx.draw_networkx_nodes(subnet, pos, node_size=500, node_color="skyblue")
nx.draw_networkx_edges(subnet, pos, width=1.0, alpha=0.5)
nx.draw_networkx_labels(subnet, pos, labels=nx.get_node_attributes(subnet, 'label'), font_size=10)
plt.title("Sottorete: Nodi con Grado >= 120")
plt.show()

In [ ]:


# Create a network interactive
net = Network(notebook=True)
net.from_nx(G)  # add the graph
net.show("grafo_interattivo.html")
# todo: open the file grafo_interattivo.html
# todo: do a better visualization

## Calculate the central node

In [ ]:
# calculate the betweenness centrality
betweenness_centrality = nx.betweenness_centrality(G)

# determine the node with the maximum betweenness centrality
central_node_betweenness = max(betweenness_centrality, key=betweenness_centrality.get)
betweenness_value = betweenness_centrality[central_node_betweenness]

# label of the node with maximum betweenness centrality
label_central_betweenness = G.nodes[central_node_betweenness]['label']

# calculate the degree centrality
degree_centrality = nx.degree_centrality(G)

# determine the node with the maximum degree centrality
central_node_degree = max(degree_centrality, key=degree_centrality.get)
degree_value = degree_centrality[central_node_degree]

# label of the node with maximum degree centrality
label_central_degree = G.nodes[central_node_degree]['label']
eigenvector_centrality = nx.eigenvector_centrality(G)

# determine the node with the maximum eigenvector centrality
central_node_eigenvector = max(eigenvector_centrality, key=eigenvector_centrality.get)
eigenvector_value = eigenvector_centrality[central_node_eigenvector]
label_central_eigenvector = G.nodes[central_node_eigenvector]['label']

# Output
print("**Centralità di Intermediazione (Betweenness Centrality)**")
print(f"Nodo centrale: {label_central_betweenness}")
print(f"Id del nodo centrale: {central_node_betweenness}")
print(f"Valore di centralità di intermediazione: {betweenness_value:.4f}\n")

print("**Centralità di Grado (Degree Centrality)**")
print(f"Nodo centrale: {label_central_degree}")
print(f"Id del nodo centrale: {central_node_degree}")
print(f"Valore di centralità di grado: {degree_value:.4f}\n")

print("**Centralità degli Autovettori (Eigenvector Centrality)**")
print(f"Nodo centrale: {label_central_eigenvector}")
print(f"Id del nodo centrale: {central_node_eigenvector}")
print(f"Valore di centralità degli autovettori: {eigenvector_value:.4f}\n")

print(f"- Il nodo '{label_central_betweenness}' ha la massima centralità di intermediazione, indicando che è un nodo chiave per connettere diverse parti del grafo.")
print(f"- Il nodo '{label_central_degree}' ha la massima centralità di grado, indicando che è il nodo con il maggior numero di connessioni dirette.")
print(f"- Il nodo '{label_central_eigenvector}' ha la massima centralità degli autovettori, indicando che è il nodo connesso a nodi importanti.")

In [ ]:
try:
    G
except NameError:
    raise RuntimeError("Il grafo G non è stato definito. Assicurati di aver eseguito il notebook precedente.")

# 1. calculate the metrics
# grde of each node
degree = dict(G.degree())
df_degree = pd.DataFrame(list(degree.items()), columns=["Nodo", "Grado"])

# Centralty
degree_centrality = nx.degree_centrality(G)
df_degree_centrality = pd.DataFrame(list(degree_centrality.items()), columns=["Nodo", "Centralità di Grado"])

# Betweenness
centrality_betweenness = nx.betweenness_centrality(G)
df_betweenness = pd.DataFrame(list(centrality_betweenness.items()), columns=["Nodo", "Centralità di Betweenness"])

# Merge the dataframes
df_metrics = (
    df_degree
    .merge(df_degree_centrality, on="Nodo")
    .merge(df_betweenness, on="Nodo")
)

# 2. order the nodes by the metrics
print("Top 20 nodi per Grado:")
print(df_metrics.nlargest(30, "Grado"))

print("\nTop 20 nodi per Centralità di Grado:")
print(df_metrics.nlargest(20, "Centralità di Grado"))

print("\nTop 20 nodi per Centralità di Betweenness:")
print(df_metrics.nlargest(20, "Centralità di Betweenness"))

# 3. show the distribution of the metrics
sns.set(style="whitegrid")

# Distribution of the degree
plt.figure(figsize=(20, 10))
sns.histplot(df_metrics["Grado"], bins=30, kde=False, color="blue")
plt.title("Distribuzione del Grado dei Nodi")
plt.xlabel("Grado")
plt.ylabel("Frequenza")
plt.show()

# Distribution of the degree centrality
plt.figure(figsize=(10, 6))
sns.histplot(df_metrics["Centralità di Grado"], bins=20, kde=False, color="green")
plt.title("Distribuzione della Centralità di Grado")
plt.xlabel("Centralità di Grado")
plt.ylabel("Frequenza")
plt.show()

# Distribution of the betweenness centrality
plt.figure(figsize=(10, 6))
sns.histplot(df_metrics["Centralità di Betweenness"], bins=20, kde=False, color="red")
plt.title("Distribuzione della Centralità di Betweenness")
plt.xlabel("Centralità di Betweenness")
plt.ylabel("Frequenza")
plt.show()


In [ ]:
# Aggiungere nodi
for _, row in nodi.iterrows():
    G.add_node(row['Id'], label=row['Label'])

# Aggiungere archi
for _, row in archi.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['Weight'])

# Calcoli per il grafo
# 1. Densità
density = nx.density(G)
print(f"Densità del grafo: {density:.4f}")

# 2. Raggio (richiede grafo connesso)
if nx.is_connected(G):
    radius = nx.radius(G)
    print(f"Raggio del grafo: {radius}")
else:
    print("Il grafo non è connesso. Calcolo sul componente connesso più grande.")
    largest_cc = max(nx.connected_components(G), key=len)
    G_largest_cc = G.subgraph(largest_cc)
    radius = nx.radius(G_largest_cc)
    print(f"Raggio del componente connesso più grande: {radius}")

# 3. Diametro (richiede grafo connesso)
if nx.is_connected(G):
    diameter = nx.diameter(G)
    print(f"Diametro del grafo: {diameter}")
else:
    diameter = nx.diameter(G_largest_cc)
    print(f"Diametro del componente connesso più grande: {diameter}")

# 4. Periferia (richiede grafo connesso)
if nx.is_connected(G):
    periphery = nx.periphery(G)
    print(f"Periferia del grafo: {periphery}")
else:
    periphery = nx.periphery(G_largest_cc)
    print(f"Periferia del componente connesso più grande: {periphery}")

# 5. Coefficiente di clustering medio
avg_clustering = nx.average_clustering(G)
print(f"Coefficiente di clustering medio: {avg_clustering:.4f}")

# 6. Connessione
is_connected = nx.is_connected(G)
print(f"Il grafo è connesso? {'Sì' if is_connected else 'No'}")

# Per i grafi non connessi, possiamo anche contare il numero di componenti connessi
num_components = nx.number_connected_components(G)
print(f"Numero di componenti connesse: {num_components}")


In [ ]:
node_file = "data/marvel-unimodal-nodes.csv"
edges_file = "data/marvel-unimodal-edges.csv"


nodes = pd.read_csv(node_file)
edges = pd.read_csv(edges_file)

# create the graph
G = nx.Graph()

# filter the nodes with degree > 50
for _, row in nodes.iterrows():
    G.add_node(row['Id'], label=row['Label'])

# add the edges
for _, row in edges.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['Weight'])

# calculate the degree for each node
degree = dict(G.degree())

# filter the nodes with degree > 50
nodes_to_keep = [node for node, deg in degree.items() if deg > 50]

# create the subgraph
G_filtered = G.subgraph(nodes_to_keep)

# create the layouts
layouts = {
    "Kamada-Kawai Layout": nx.kamada_kawai_layout(G_filtered),
    "Spiral Layout": nx.spiral_layout(G_filtered),
    "Spring Layout": nx.spring_layout(G_filtered),
    "Circular Layout": nx.circular_layout(G_filtered)
}

# plot the graph
plt.figure(figsize=(20, 20))  # Dim of the plot
for i, (layout_name, layout_pos) in enumerate(layouts.items(), 1):
    plt.subplot(2, 2, i)  # Grid 2x2
    nx.draw(
        G_filtered,
        pos=layout_pos,  
        with_labels=True,  
        node_size=500,  
        node_color="skyblue",  
        font_size=10,  
        edge_color="gray",  
        alpha=0.7  
    )
    plt.title(layout_name, fontsize=15)  #

plt.tight_layout()
plt.show()
